In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import time

con = sqlite3.connect('../summer')
cur = con.cursor()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
recordings = ['\'week1_3\'', '\'week2_3\'', '\'week3_3\'', '\'week4_3\'', '\'week5_3\'', '\'week6_3\'']
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']


all_weeks = pd.read_sql_query("""SELECT PlayerName, 
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings)), con)

playerIdx = {
    'Alonzo': 0,
    'Dax': 1,
    'Emily': 2,
    'Isabelle': 3,
    'Jake': 4,
    'Philip': 5,
    'Rishab': 6,
    'Sam': 7,
    'justin': 8,
    'Alex': 9,
    'Jordan': 10
}

motion_output = []
def create_labels(row):
    output = np.zeros(11)
    output[playerIdx[row['PlayerName']]] = 1
    motion_output.append(output)

all_weeks.apply(lambda x: create_labels(x), axis=1)
training_y = np.asarray(motion_output)

all_weeks = all_weeks.drop(columns=['PlayerName'])
training_x = np.asarray(all_weeks)
        
print(training_x.shape)
print(training_y.shape)

(3189974, 21)
(3189974, 11)


In [3]:
recordings_test = ['\'week7_3\'', '\'week8_3\'']
students = ['\'Alonzo\'','\'Dax\'','\'Emily\'','\'Isabelle\'','\'Jake\'','\'Philip\'','\'Rishab\'','\'Sam\'',
            '\'justin\'','\'Alex\'','\'Jordan\'']


all_weeks_test = pd.read_sql_query("""SELECT PlayerName, 
    HeadPos_x, HeadPos_y, HeadPos_z, 
    LeftHandPos_x, LeftHandPos_y, LeftHandPos_z,
    RightHandPos_x, RightHandPos_y, RightHandPos_z,
    HeadRot_w, HeadRot_x, HeadRot_y, HeadRot_z,
    LeftHandRot_w, LeftHandRot_x, LeftHandRot_y, LeftHandRot_z,
    RightHandRot_w, RightHandRot_x, RightHandRot_y, RightHandRot_z
    FROM PersonTime
    WHERE PlayerName IN ({})
    AND RecordingName IN ({})
    """.format(",".join(students), ",".join(recordings_test)), con)

motion_output = []
all_weeks_test.apply(lambda x: create_labels(x), axis=1)
testing_y = np.asarray(motion_output)

all_weeks_test = all_weeks_test.drop(columns=['PlayerName'])
testing_x = np.asarray(all_weeks_test)

print(testing_x.shape)
print(testing_y.shape)

(919446, 21)
(919446, 11)


In [4]:
class Data:

    def __init__(self, training_X, training_y, testing_X, testing_y, num_examples, input_dim, output_dim):
        self.training_X = training_X
        self.training_y = training_y
        self.testing_X = testing_X
        self.testing_y = testing_y
        self.num_examples = num_examples
        self.index_in_epoch = 0
        self.epochs_completed = 0
        self.input_dim = input_dim
        self.output_dim = output_dim

    def next_batch(self, batch_size):
        start = self.index_in_epoch
        self.index_in_epoch += batch_size
        if self.index_in_epoch > self.num_examples:
            self.epochs_completed += 1
            perm = np.arange(self.num_examples)
            np.random.shuffle(perm)
            self.training_X = self.training_X[perm]
            self.training_y = self.training_y[perm]
            start = 0
            self.index_in_epoch = batch_size
            assert batch_size <= self.num_examples
        end = self.index_in_epoch
        return self.training_X[start:end], self.training_y[start:end]

In [5]:
class MotionData(Data):

    def __init__(self):
        Data.__init__(self, training_x, training_y, testing_x, testing_y, 3189974, 21, 11)

data = MotionData()

In [6]:
class ClassificationNN:
    def __init__(self,dataset,learning_rate,training_iteration,batch_size, hidden_layer_n):
        self.dataset = data

        self.learning_rate = learning_rate
        self.training_iteration = training_iteration
        self.batch_size = batch_size
        input_layer_n = self.dataset.input_dim
        output_layer_n = self.dataset.output_dim
        self.display_step = 3
        
        tf.reset_default_graph()

        self.x = tf.placeholder("float", [None, input_layer_n])
        self.y = tf.placeholder("float", [None, output_layer_n])

        W1 = tf.Variable(tf.random_normal([input_layer_n, hidden_layer_n], stddev=0.1), name='W1')
        W2 = tf.Variable(tf.random_normal([hidden_layer_n, output_layer_n], stddev=0.1), name='W2')

        b1 = tf.Variable(tf.random_normal([hidden_layer_n], stddev=0.1), name='b1')
        b2 = tf.Variable(tf.random_normal([output_layer_n], stddev=0.1), name='b2')

        layer_1 = tf.nn.sigmoid(tf.matmul(self.x, W1) + b1)
        layer_2 = tf.nn.softmax(tf.matmul(layer_1, W2) + b2)
        self.model = layer_2

        self.objective_function = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.model, self.y))))

        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.objective_function)

        self.init = tf.global_variables_initializer()
        
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            print('start training: ', time.strftime("%H:%M:%S", time.localtime()))
            sess.run(self.init)
            for iteration in range(self.training_iteration):
                avg_cost = 0
                num_batches = int(self.dataset.num_examples/self.batch_size)
                for i in range(num_batches):
                    batch_xs, batch_ys = self.dataset.next_batch(self.batch_size)
                    sess.run(self.optimizer, feed_dict={self.x: batch_xs, self.y: batch_ys})
                    avg_cost += sess.run(self.objective_function, feed_dict={self.x: batch_xs, self.y: batch_ys})/num_batches
                if iteration % self.display_step == 0:
                    print("Epoch:" + str(iteration+1) +  " cost: ", "{:.9f}".format(avg_cost))

            print("Training completed!", time.strftime("%H:%M:%S", time.localtime()))
  
            predictions = tf.equal(tf.argmax(self.model, 1), tf.argmax(self.y, 1))
            accuracy = tf.reduce_mean(tf.cast(predictions,  tf.float64))
            print("Testing accuracy:", accuracy.eval({self.x: self.dataset.testing_X, self.y: self.dataset.testing_y}))
            
            end_time = str(time.strftime("%d_%H_%M_%S", time.localtime()))
            os.makedirs('models_basic_nn/' + end_time)
            
            np.savetxt('models_basic_nn/' + end_time + 'W1.csv', W1.eval())
            np.savetxt('models_basic_nn/' + end_time + 'W2.csv', W2.eval())
            np.savetxt('models_basic_nn/' + end_time + 'b1.csv', b1.eval())
            np.savetxt('models_basic_nn/' + end_time + 'b2.csv', b2.eval())
            
            print(saver._var_list)
            saver.save(sess, 'models_basic_nn/' + end_time + '/basic_nn_' + end_time)

In [7]:
nn = ClassificationNN(data, learning_rate=0.015,batch_size=50,training_iteration=100, hidden_layer_n=200)

start training:  17:22:41


2022-04-19 17:22:41.957990: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch:1 cost:  4.603098633
Epoch:4 cost:  4.381387493
Epoch:7 cost:  3.522129298
Epoch:10 cost:  3.268951338
Epoch:13 cost:  3.168045434
Epoch:16 cost:  3.056314927
Epoch:19 cost:  2.913836595
Epoch:22 cost:  2.800207165
Epoch:25 cost:  2.872795140
Epoch:28 cost:  2.898790318
Epoch:31 cost:  2.969932063
Epoch:34 cost:  2.852505961
Epoch:37 cost:  2.922426345
Epoch:40 cost:  2.828262506
Epoch:43 cost:  2.704886043
Epoch:46 cost:  2.758890736
Epoch:49 cost:  2.675928972
Epoch:52 cost:  2.670455876
Epoch:55 cost:  2.829352537
Epoch:58 cost:  2.732609188
Epoch:61 cost:  2.759842108
Epoch:64 cost:  2.979058686
Epoch:67 cost:  2.898624289
Epoch:70 cost:  2.942134273
Epoch:73 cost:  2.799542480
Epoch:76 cost:  2.764407479
Epoch:79 cost:  2.724114043
Epoch:82 cost:  2.636562307
Epoch:85 cost:  2.631493709
Epoch:88 cost:  2.619046361
Epoch:91 cost:  2.562664514
Epoch:94 cost:  2.704714945
Epoch:97 cost:  2.927868365
Epoch:100 cost:  2.766196925
Training completed! 19:02:30
Testing accuracy: 0.3